In [ ]:
!git clone https://github.com/itsaliaze/SMAP-Downscaling.git

Installing Libraries

In [ ]:
# Installing necessary libraries for geospatial data handling
!pip install geopandas geojson fiona georasters rioxarray xarray raster2xyz rasterio kerastuner

# Importing required libraries
import fiona, geojson, folium, ee, numpy as np
import geopandas as gpd, georasters as gr, matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
import geemap

Initializing GEE

In [ ]:
# Initialize Google Earth Engine (GEE)
ee.Authenticate()  # Authenticate GEE access
ee.Initialize(project='august-edge-339813')  # Initialize GEE project
geemap.ee_initialize(project='august-edge-339813')  # Initialize geemap

Period of Interest

In [ ]:
# Period of interest
start_date = '2023-11-03'
end_date = '2023-12-05'

Uploading and Plotting the Study Area Shapefile

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Read the shapefile
CV = gpd.read_file("/content/Alluvial_Bnd.shp")

# Plot the shapefile without the surrounding box
fig, ax = plt.subplots()
CV.plot(ax=ax)

# Remove the surrounding box
ax.set_frame_on(False)

# Remove x-axis and y-axis tick marks
ax.set_xticks([])
ax.set_yticks([])

# Show the plot
plt.show()

Converting Shapefile to FeatureCollection and Extracting Geometry

In [ ]:
# Convert the GeoDataFrame to an ee.FeatureCollection
fc = geemap.geopandas_to_ee(CV)
# Get the first feature from the FeatureCollection
feature_ = ee.Feature(fc.first())
# Extract the geometry from the feature
CV = feature_.geometry()

Defining Predictor Variables

In [ ]:
# SMAP Surface Soil Moisture
SMAP = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
    .filterDate(start_date, end_date) \
    .select('sm_surface').mean().clip(CV)

# SAR Imagery (VH & VV Polarization)
VH = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(start_date, end_date).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).select('VH') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')).mean().clip(CV)
VV = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(start_date, end_date).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).select('VV') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')).mean().clip(CV)

# Precipitation (Total Precipitation during the period)
CHIRPS = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate(start_date, end_date).sum().select('precipitation').clip(CV)

# Land Surface Temperature (Landsat)
LST = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(start_date, end_date).mean().select('ST_B10').clip(CV)

# Brightness Temperature
BT = ee.ImageCollection('NOAA/CDR/PATMOSX/V53') \
    .filterDate(start_date, end_date).mean().select('temp_12_0um').clip(CV)

# MODIS Daytime Temperature
Temp = ee.ImageCollection('MODIS/061/MOD11A2') \
    .filterDate(start_date, end_date).mean().select('LST_Day_1km').clip(CV)

# Evapotranspiration (MODIS)
ET = ee.ImageCollection('MODIS/061/MOD16A2') \
    .filterDate(start_date, end_date).mean().select('ET').clip(CV)

# Elevation (DEM)
DEM = ee.Image('USGS/3DEP/10m').select('elevation').clip(CV)

# Sentinel-2
# Define a function to mask clouds using the Sentinel-2 QA60 band
def mask_s2_clouds(image):
    # Use QA60 band to mask out clouds (1 indicates cloud, 0 indicates no cloud)
    cloud_mask = image.select('QA60').eq(0)
    return image.updateMask(cloud_mask)
# Sentinel-2 with cloud filtering based on CLOUDY_PIXEL_PERCENTAGE
Sentinel2 = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterDate(start_date, end_date)  # Define the date range
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  # Keep images with less than 20% cloud cover
    .select('B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B11', 'B12')  # Select relevant bands
    .map(mask_s2_clouds)  # Apply cloud mask
    .mean()  # Compute the mean of the images
    .clip(CV)  # Clip to the study area
)

# Cropland Data Layer (USDA)
Crop = ee.ImageCollection('USDA/NASS/CDL').filterDate(start_date, end_date).select('cropland').first().clip(CV)

# Land Cover (Dynamic World)
LC = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).select('label').mean().clip(CV)

# Slope (from DEM)
SLOPE = ee.Terrain.slope(DEM).clip(CV)

# Topographic Roughness (Standard deviation of DEM)
ROUGHNESS = DEM.reduceNeighborhood(ee.Reducer.stdDev(), ee.Kernel.square(5)).clip(CV)

#Soil Properties
silt = ee.ImageCollection("projects/sat-io/open-datasets/polaris/silt_mean").first().clip(CV)
clay = ee.ImageCollection("projects/sat-io/open-datasets/polaris/clay_mean").first().clip(CV)
sand = ee.ImageCollection("projects/sat-io/open-datasets/polaris/sand_mean").first().clip(CV)
om = ee.ImageCollection('projects/sat-io/open-datasets/polaris/om_mean').first().clip(CV)
bd = ee.ImageCollection('projects/sat-io/open-datasets/polaris/bd_mean').first().clip(CV)

# Topographic Wetness Index (TWI)
cell_size = 10.2 # Define cell size (resolution) in meters
weights = ee.List.repeat(ee.List.repeat(1, 3), 3)  # Define a 3x3 kernel
kernel = ee.Kernel.fixed(3, 3, weights)  # Create fixed kernel
flow_accumulation = SLOPE.gt(0).reduceNeighborhood(
    reducer=ee.Reducer.sum(),
    kernel=kernel
)
TWI = flow_accumulation.log().divide(SLOPE.tan())

# Function to calculate vegetation indices for each image
def calculate_indices(image):
    # NDVI, EVI, SAVI, NDWI, LAI calculation
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = image.expression('2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))', {
        'NIR': image.select('B8'),
        'Red': image.select('B4'),
        'Blue': image.select('B2')
    }).rename('EVI')
    savi = image.expression('(1 + L) * ((NIR - Red) / (NIR + Red + L))', {
        'NIR': image.select('B8'),
        'Red': image.select('B4'),
        'L': 0.5
    }).rename('SAVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    lai = image.expression('3.618 * ((NIR - Red) / (0.5 + NIR + Red))', {
        'NIR': image.select('B8'),
        'Red': image.select('B4')
    }).rename('LAI')
    return image.addBands([ndvi, evi, savi, ndwi, lai])

# Apply the vegetation indices calculation to Sentinel-2 imagery
S2 = calculate_indices(Sentinel2)

# Extract individual indices
NDVI = S2.select('NDVI')
EVI = S2.select('EVI')
SAVI = S2.select('SAVI')
NDWI = S2.select('NDWI')
LAI = S2.select('LAI')

Display All Variables in GEEMap

In [ ]:
import geemap

# Create a geemap instance
Map = geemap.Map()

# Add the Central Valley Geometry object to the map
Map.addLayer(CV, {}, 'ROI', True)

# Add the SMAP layer
Map.addLayer(SMAP, {'min': 0, 'max': 1}, 'SMAP', True)

# Add the SAR imagery (VH and VV)
Map.addLayer(VH, {'min': -50, 'max': 0}, 'VH', True)
Map.addLayer(VV, {'min': -50, 'max': 0}, 'VV', True)

# Add the precipitation imagery (CHIRPS)
Map.addLayer(CHIRPS, {'min': 0, 'max': 40}, 'Precipitation', True)

# Add the Land Surface Temperature (LST)
Map.addLayer(LST, {'min': 0, 'max': 83323}, 'Land Surface Temperature', True)

# Add the Evapotranspiration (ET)
Map.addLayer(ET, {'min': 20, 'max': 150}, 'Evapotranspiration', True)

# Add the Sentinel-2 imagery with bands and indices
Map.addLayer(S2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3500}, 'Sentinel-2', True)

# Add the Digital Elevation Model (DEM)
Map.addLayer(DEM, {'min': 0, 'max': 4500}, 'DEM', True)

# Add the Slope image
Map.addLayer(SLOPE, {'min': 0, 'max': 40}, 'SLOPE', True)

# Add the TWI (Topographic Wetness Index)
Map.addLayer(TWI, {'min': -3, 'max': 30}, 'TWI', True)

# Add the Roughness layer
Map.addLayer(ROUGHNESS, {'min': 0, 'max': 800}, 'ROUGHNESS', True)

# Set visualization parameters for the vegetation indices
vis_params_indexes = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green', 'yellow', 'blue']}

# Add the vegetation indices to the map
Map.addLayer(S2.select('NDVI'), vis_params_indexes, 'NDVI', True)
Map.addLayer(S2.select('EVI'), vis_params_indexes, 'EVI', True)
Map.addLayer(S2.select('SAVI'), vis_params_indexes, 'SAVI', True)
Map.addLayer(S2.select('NDWI'), vis_params_indexes, 'NDWI', True)
Map.addLayer(S2.select('LAI'), vis_params_indexes, 'LAI', True)

# Add soil properties (e.g., sand, silt, clay, organic matter, and bulk density)
Map.addLayer(sand, {'min': 5, 'max': 90}, 'Sand', True)
Map.addLayer(silt, {'min': 2, 'max': 80}, 'Silt', True)
Map.addLayer(clay, {'min': 3, 'max': 55}, 'Clay', True)
Map.addLayer(om, {'min': -0.8, 'max': 1.8}, 'Organic Matter', True)
Map.addLayer(bd, {'min': 0.6, 'max': 1.6}, 'Bulk Density', True)

# Add the crop layer
Map.addLayer(Crop, {'min': 0, 'max': 254}, 'Crop', True)

# Center the map around California's Central Valley
Map.centerObject(CV, zoom=10)

# Display the map
Map

DATA PRE-PROCESSING

Resampling data to different resolutions (1000 × 1000 m, 100 × 100 m, 50 × 50 m) and the target resolution (30 × 30 m) to prepare the training and contextual data for the downscaling model.

In [ ]:
# Function to reproject and resample the raster layers
def resample_to_resolution(image, resolution, projection='EPSG:4326'):
    """
    This function reprojects and resamples a given image to the desired resolution
    using bicubic resampling.

    Parameters:
    - image: The image to be resampled.
    - resolution: The target resolution (in meters).
    - projection: The projection system to use (default is 'EPSG:4326').

    Returns:
    - Resampled image.
    """
    return image.reproject(crs=projection, scale=resolution).resample('bicubic').reduceResolution(reducer=ee.Reducer.mean(), maxPixels=10240)

# Selecting the common spatial resolution for all the variables
resolution = 1000
# Selecting the common projection system (EPSG:4326)
projection = 'EPSG:4326'

In [ ]:
# Reproject and resample the variable rasters to the desired resolution
SMAP_1000m = resample_to_resolution(SMAP, resolution)
VH_1000m = resample_to_resolution(VH, resolution)
VV_1000m = resample_to_resolution(VV, resolution)
CHIRPS_1000m = resample_to_resolution(CHIRPS, resolution)
ET_1000m = resample_to_resolution(ET, resolution)
DEM_1000m = resample_to_resolution(DEM, resolution)
SLOPE_1000m = resample_to_resolution(SLOPE, resolution)
TWI_1000m = resample_to_resolution(TWI, resolution)
LST_1000m = resample_to_resolution(LST, resolution)
NDVI_1000m = resample_to_resolution(S2.select('NDVI'), resolution)
EVI_1000m = resample_to_resolution(S2.select('EVI'), resolution)
SAVI_1000m = resample_to_resolution(S2.select('SAVI'), resolution)
NDWI_1000m = resample_to_resolution(S2.select('NDWI'), resolution)
LAI_1000m = resample_to_resolution(S2.select('LAI'), resolution)
Crop_1000m = resample_to_resolution(Crop, resolution)
LC_1000m = resample_to_resolution(LC, resolution)
SAND_1000m = resample_to_resolution(sand, resolution)
SILT_1000m = resample_to_resolution(silt, resolution)
CLAY_1000m = resample_to_resolution(clay, resolution)
BD_1000m = resample_to_resolution(bd, resolution)
OM_1000m = resample_to_resolution(om, resolution)

Exporting the data

In [ ]:
# List of all predictor variables
predictors_1000m = [
    SMAP_1000m, VH_1000m, VV_1000m, CHIRPS_1000m, ET_1000m, DEM_1000m,
    SLOPE_1000m, TWI_1000m, LST_1000m, NDVI_1000m, EVI_1000m, SAVI_1000m,
    NDWI_1000m, LAI_1000m, Crop_1000m, LC_1000m, SAND_1000m, SILT_1000m,
    CLAY_1000m, BD_1000m, OM_1000m
]

# Export each variable to Google Drive
for idx, variable in enumerate(predictors_100m):
    task = ee.batch.Export.image.toDrive(
        image=variable,
        description=f'export_{variable.getInfo()["id"]}',  # Unique description for each variable
        folder='SMAP_DOWNSCALING',
        region=CV,
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )

    task.start()  # Start the export task
    print(f"Exporting {variable.getInfo()['id']}...")  # Inform the user which variable is being exported

# Monitor the export status
for task in ee.batch.Task.list():
    print(task.status())

Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Preparing the df for a specific resolution

In [ ]:
import pandas as pd
from raster2xyz.raster2xyz import Raster2xyz

# Initialize Raster2xyz object
rtxyz = Raster2xyz()

# Convert each raster to CSV and load as DataFrame

# SMAP 1000m
input_raster_smap_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SMAP_D_1000m.tif'
output_csv_smap_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SMAP_D_1000m.csv'
rtxyz.translate(input_raster_smap_1000m, output_csv_smap_1000m)
smap_raster_df_1000m = pd.read_csv(output_csv_smap_1000m, skiprows=1, names=['x', 'y', 'SMAP']).dropna()

# VH 1000m
input_raster_vh_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/VH_1000m.tif'
output_csv_vh_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/VH_1000m.csv'
rtxyz.translate(input_raster_vh_1000m, output_csv_vh_1000m)
vh_raster_df_1000m = pd.read_csv(output_csv_vh_1000m, skiprows=1, names=['x', 'y', 'VH']).dropna()

# VV 1000m
input_raster_vv_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/VV_1000m.tif'
output_csv_vv_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/VV_1000m.csv'
rtxyz.translate(input_raster_vv_1000m, output_csv_vv_1000m)
vv_raster_df_1000m = pd.read_csv(output_csv_vv_1000m, skiprows=1, names=['x', 'y', 'VV']).dropna()

# CHIRPS 1000m
input_raster_chirps_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/CHIRPS_1000m.tif'
output_csv_chirps_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/CHIRPS_1000m.csv'
rtxyz.translate(input_raster_chirps_1000m, output_csv_chirps_1000m)
chirps_raster_df_1000m = pd.read_csv(output_csv_chirps_1000m, skiprows=1, names=['x', 'y', 'CHIRPS']).dropna()

# LST 1000m
input_raster_lst_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/LST_1000m.tif'
output_csv_lst_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/LST_1000m.csv'
rtxyz.translate(input_raster_lst_1000m, output_csv_lst_1000m)
lst_raster_df_1000m = pd.read_csv(output_csv_lst_1000m, skiprows=1, names=['x', 'y', 'LST']).dropna()

# BT 1000m
input_raster_bt_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/BT_1000m.tif'
output_csv_bt_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/BT_1000m.csv'
rtxyz.translate(input_raster_bt_1000m, output_csv_bt_1000m)
bt_raster_df_1000m = pd.read_csv(output_csv_bt_1000m, skiprows=1, names=['x', 'y', 'BT']).dropna()

# ET 1000m
input_raster_et_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/ET_1000m.tif'
output_csv_et_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/ET_1000m.csv'
rtxyz.translate(input_raster_et_1000m, output_csv_et_1000m)
et_raster_df_1000m = pd.read_csv(output_csv_et_1000m, skiprows=1, names=['x', 'y', 'ET']).dropna()

# DEM 1000m
input_raster_dem_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/DEM_1000m.tif'
output_csv_dem_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/DEM_1000m.csv'
rtxyz.translate(input_raster_dem_1000m, output_csv_dem_1000m)
dem_raster_df_1000m = pd.read_csv(output_csv_dem_1000m, skiprows=1, names=['x', 'y', 'DEM']).dropna()

# SLOPE 1000m
input_raster_slope_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SLOPE_1000m.tif'
output_csv_slope_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SLOPE_1000m.csv'
rtxyz.translate(input_raster_slope_1000m, output_csv_slope_1000m)
slope_raster_df_1000m = pd.read_csv(output_csv_slope_1000m, skiprows=1, names=['x', 'y', 'SLOPE']).dropna()

# TWI 1000m
input_raster_twi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/TWI_1000m.tif'
output_csv_twi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/TWI_1000m.csv'
rtxyz.translate(input_raster_twi_1000m, output_csv_twi_1000m)
twi_raster_df_1000m = pd.read_csv(output_csv_twi_1000m, skiprows=1, names=['x', 'y', 'TWI']).dropna()

# NDVI 1000m
input_raster_ndvi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/NDVI_1000m.tif'
output_csv_ndvi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/NDVI_1000m.csv'
rtxyz.translate(input_raster_ndvi_1000m, output_csv_ndvi_1000m)
ndvi_raster_df_1000m = pd.read_csv(output_csv_ndvi_1000m, skiprows=1, names=['x', 'y', 'NDVI']).dropna()

# EVI 1000m
input_raster_evi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/EVI_1000m.tif'
output_csv_evi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/EVI_1000m.csv'
rtxyz.translate(input_raster_evi_1000m, output_csv_evi_1000m)
evi_raster_df_1000m = pd.read_csv(output_csv_evi_1000m, skiprows=1, names=['x', 'y', 'EVI']).dropna()

# SAVI 1000m
input_raster_savi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SAVI_1000m.tif'
output_csv_savi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SAVI_1000m.csv'
rtxyz.translate(input_raster_savi_1000m, output_csv_savi_1000m)
savi_raster_df_1000m = pd.read_csv(output_csv_savi_1000m, skiprows=1, names=['x', 'y', 'SAVI']).dropna()

# NDWI 1000m
input_raster_ndwi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/NDWI_1000m.tif'
output_csv_ndwi_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/NDWI_1000m.csv'
rtxyz.translate(input_raster_ndwi_1000m, output_csv_ndwi_1000m)
ndwi_raster_df_1000m = pd.read_csv(output_csv_ndwi_1000m, skiprows=1, names=['x', 'y', 'NDWI']).dropna()

# LAI 1000m
input_raster_lai_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/LAI_1000m.tif'
output_csv_lai_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/LAI_1000m.csv'
rtxyz.translate(input_raster_lai_1000m, output_csv_lai_1000m)
lai_raster_df_1000m = pd.read_csv(output_csv_lai_1000m, skiprows=1, names=['x', 'y', 'LAI']).dropna()

# Crop 1000m
input_raster_crop_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/Crop_1000m.tif'
output_csv_crop_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/Crop_1000m.csv'
rtxyz.translate(input_raster_crop_1000m, output_csv_crop_1000m)
crop_raster_df_1000m = pd.read_csv(output_csv_crop_1000m, skiprows=1, names=['x', 'y', 'Crop']).dropna()

# LC 1000m
input_raster_lc_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/LC_1000m.tif'
output_csv_lc_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/LC_1000m.csv'
rtxyz.translate(input_raster_lc_1000m, output_csv_lc_1000m)
lc_raster_df_1000m = pd.read_csv(output_csv_lc_1000m, skiprows=1, names=['x', 'y', 'LC']).dropna()

# SAND 1000m
input_raster_sand_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SAND_1000m.tif'
output_csv_sand_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SAND_1000m.csv'
rtxyz.translate(input_raster_sand_1000m, output_csv_sand_1000m)
sand_raster_df_1000m = pd.read_csv(output_csv_sand_1000m, skiprows=1, names=['x', 'y', 'SAND']).dropna()

# SILT 1000m
input_raster_silt_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SILT_1000m.tif'
output_csv_silt_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/SILT_1000m.csv'
rtxyz.translate(input_raster_silt_1000m, output_csv_silt_1000m)
silt_raster_df_1000m = pd.read_csv(output_csv_silt_1000m, skiprows=1, names=['x', 'y', 'SILT']).dropna()

# CLAY 1000m
input_raster_clay_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/CLAY_1000m.tif'
output_csv_clay_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/CLAY_1000m.csv'
rtxyz.translate(input_raster_clay_1000m, output_csv_clay_1000m)
clay_raster_df_1000m = pd.read_csv(output_csv_clay_1000m, skiprows=1, names=['x', 'y', 'CLAY']).dropna()

# BD 1000m
input_raster_bd_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/BD_1000m.tif'
output_csv_bd_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/BD_1000m.csv'
rtxyz.translate(input_raster_bd_1000m, output_csv_bd_1000m)
bd_raster_df_1000m = pd.read_csv(output_csv_bd_1000m, skiprows=1, names=['x', 'y', 'BD']).dropna()

# OM 1000m
input_raster_om_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/OM_1000m.tif'
output_csv_om_1000m = '/content/drive/MyDrive/SMAP_DOWNSCALING_CV/OM_1000m.csv'
rtxyz.translate(input_raster_om_1000m, output_csv_om_1000m)
om_raster_df_1000m = pd.read_csv(output_csv_om_1000m, skiprows=1, names=['x', 'y', 'OM']).dropna()

Merging Dfs

In [ ]:
final_df_1000m = (
    smap_raster_df_1000m
    .merge(vh_raster_df_1000m, on=['x', 'y'])
    .merge(vv_raster_df_1000m, on=['x', 'y'])
    .merge(chirps_raster_df_1000m, on=['x', 'y'])
    .merge(et_raster_df_1000m, on=['x', 'y'])
    .merge(dem_raster_df_1000m, on=['x', 'y'])
    .merge(slope_raster_df_1000m, on=['x', 'y'])
    .merge(twi_raster_df_1000m, on=['x', 'y'])
    .merge(lst_raster_df_1000m, on=['x', 'y'])
    .merge(ndvi_raster_df_1000m, on=['x', 'y'])
    .merge(evi_raster_df_1000m, on=['x', 'y'])
    .merge(savi_raster_df_1000m, on=['x', 'y'])
    .merge(ndwi_raster_df_1000m, on=['x', 'y'])
    .merge(lai_raster_df_1000m, on=['x', 'y'])
    .merge(crop_raster_df_1000m, on=['x', 'y'])
    .merge(lc_raster_df_1000m, on=['x', 'y'])
    .merge(sand_raster_df_1000m, on=['x', 'y'])
    .merge(silt_raster_df_1000m, on=['x', 'y'])
    .merge(clay_raster_df_1000m, on=['x', 'y'])
    .merge(bd_raster_df_1000m, on=['x', 'y'])
    .merge(om_raster_df_1000m, on=['x', 'y'])
)

# Output the shape of the final DataFrame and ensure numeric conversion
print(final_df_1000m.shape)

# Ensure the 'SMAP' column is numeric, coercing any non-numeric values to NaN
final_df_1000m['SMAP'] = pd.to_numeric(final_df_1000m['SMAP'], errors='coerce')

# Display the first few rows of the final DataFrame
final_df_1000m.head()

Concatenation of all the resolutions

In [ ]:
# Concatenate all dataframes by adding new rows
final_combined_df = pd.concat([final_df_1000m, final_df_100m, final_df_50m], ignore_index=True)

# Output the shape of the final combined DataFrame
print(final_combined_df.shape)

# Display the first few rows of the combined DataFrame
final_combined_df.head()

Standardize the data

In [ ]:
# Exclude 'x' and 'y' columns from the DataFrame
columns_to_exclude = ['x', 'y']
df_filtered = final_combined_df.drop(columns=columns_to_exclude)

# Standardize the data
scaler = StandardScaler()
df_final = pd.DataFrame(scaler.fit_transform(df_filtered), columns=df_filtered.columns)

Training and testing data

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import numpy as np

# Prepare the X and y
X = df_final[['VH', 'VV', 'CHIRPS', 'ET', 'DEM', 'SLOPE', 'TWI', 'LST', 'NDVI', 'SAVI', 'NDWI', 'LAI', 'Crop', 'LC', 'EVI', 'OM', 'BD', 'Sand', 'Silt', 'Clay']].values
y = df_final[['SMAP']].values

# KFold Cross-validation setup
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize arrays to store the results for each fold (loss and MSE)
dnn_loss, dnn_mse = [], []
cnn_loss, cnn_mse = [], []
lstm_loss, lstm_mse = [], []
ensemble_dnn_loss, ensemble_dnn_mse = [], []

Hyperparameters optimization

In [ ]:
# Define the DNN model function
def build_dnn_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 1, 16)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=1024, step=32),
                        activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'elu', 'softmax', 'exponential'])))
        model.add(Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

# Define the CNN model function
def build_cnn_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_convolutions', 1, 4)):
        model.add(Conv1D(filters=hp.Int('filters_' + str(i), min_value=32, max_value=256, step=16),
                         kernel_size=hp.Int('kernel_size', 3, 5, step=1),
                         activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'elu', 'softmax', 'exponential']),
                         input_shape=(X_train.shape[1], 1) if i == 0 else None))
        model.add(MaxPooling1D(pool_size=hp.Int('pooling_size', 2, 4, step=1)))
    model.add(Flatten())
    for j in range(hp.Int('num_dense_layers', 1, 16)):
        model.add(Dense(units=hp.Int('dense_nodes_' + str(j), min_value=32, max_value=1024, step=32),
                        activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'elu', 'softmax', 'exponential'])))
        model.add(Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

# Define the LSTM model function
def build_lstm_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_lstm_layers', 1, 16)):
        model.add(LSTM(units=hp.Int('units_' + str(i), min_value=32, max_value=256, step=16),
                       return_sequences=(i < hp.Int('num_lstm_layers', 1, 16) - 1)))
        model.add(Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

# Define the Ensemble DNN model function
def build_ensemble_dnn_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 1, 16)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=1024, step=32),
                        activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'elu', 'softmax', 'exponential'])))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

Defining search space

In [ ]:
import kerastuner as kt

#DNN tuning
tuner_dnn = kt.BayesianOptimization(
    build_dnn_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=1,
    directory='md',
    project_name='dnn_bayesian_optimization'
)

#CNN tuning
tuner_cnn = kt.BayesianOptimization(
    build_cnn_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=1,
    directory='md',
    project_name='cnn_bayesian_optimization'
)

#LSTM tuning
tuner_lstm = kt.BayesianOptimization(
    build_lstm_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=1,
    directory='md',
    project_name='lstm_bayesian_optimization'
)

#Ensemble DNN tuning
tuner_ensemble_dnn = kt.BayesianOptimization(
    build_ensemble_dnn_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=1,
    directory='md',
    project_name='ensemble_dnn_bayesian_optimization'
)

In [ ]:
# Loop over the 10 folds for cross-validation
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Reshape y_train and y_val
    y_train = y_train.reshape(-1)
    y_val = y_val.reshape(-1)

In [ ]:
    # Run the hyperparameter search for DNN
    tuner_dnn.search(X_train, y_train, epochs=100, validation_data=(X_val, y_val))
    best_dnn_model = tuner_dnn.get_best_models(num_models=1)[0]
    loss_dnn, mse_dnn = best_dnn_model.evaluate(X_val, y_val)
    dnn_loss.append(loss_dnn)
    dnn_mse.append(mse_dnn)

    # Run the hyperparameter search for CNN
    tuner_cnn.search(X_train, y_train, epochs=100, validation_data=(X_val, y_val))
    best_cnn_model = tuner_cnn.get_best_models(num_models=1)[0]
    loss_cnn, mse_cnn = best_cnn_model.evaluate(X_val, y_val)
    cnn_loss.append(loss_cnn)
    cnn_mse.append(mse_cnn)

    # Run the hyperparameter search for LSTM
    tuner_lstm.search(X_train, y_train, epochs=100, validation_data=(X_val, y_val))
    best_lstm_model = tuner_lstm.get_best_models(num_models=1)[0]
    loss_lstm, mse_lstm = best_lstm_model.evaluate(X_val, y_val)
    lstm_loss.append(loss_lstm)
    lstm_mse.append(mse_lstm)

    # Run the hyperparameter search for Ensemble DNN
    tuner_ensemble_dnn.search(X_train, y_train, epochs=100, validation_data=(X_val, y_val))
    best_ensemble_dnn_model = tuner_ensemble_dnn.get_best_models(num_models=1)[0]
    loss_ensemble_dnn, mse_ensemble_dnn = best_ensemble_dnn_model.evaluate(X_val, y_val)
    ensemble_dnn_loss.append(loss_ensemble_dnn)
    ensemble_dnn_mse.append(mse_ensemble_dnn)

# After all the folds, calculate average loss and mse for each model
print(f"DNN - Average Loss: {np.mean(dnn_loss)}")
print(f"DNN - Average MSE: {np.mean(dnn_mse)}")

print(f"CNN - Average Loss: {np.mean(cnn_loss)}")
print(f"CNN - Average MSE: {np.mean(cnn_mse)}")

print(f"LSTM - Average Loss: {np.mean(lstm_loss)}")
print(f"LSTM - Average MSE: {np.mean(lstm_mse)}")

print(f"Ensemble DNN - Average Loss: {np.mean(ensemble_dnn_loss)}")
print(f"Ensemble DNN - Average MSE: {np.mean(ensemble_dnn_mse)}")

Final model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

def build_dnn_model():
    model = Sequential()
    activations = ['elu', 'relu', 'tanh', 'softmax']
    for i in range(17):
        model.add(Dense(64, activation=activations[i % len(activations)]))
        model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=160, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

def build_lstm_model(input_shape):
    model = Sequential()
    for _ in range(7):
        model.add(LSTM(32, return_sequences=True, dropout=0.15, input_shape=input_shape))
    model.add(LSTM(32))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

def build_ensemble_dnn_model():
    model = Sequential()
    activations = ['relu', 'tanh', 'softmax']
    for i in range(6):
        model.add(Dense(64, activation=activations[i % len(activations)]))
        model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_reshaped = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

# Train Base DNN Model
dnn_model = build_dnn_model()
dnn_history = dnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Train CNN Model
cnn_model = build_cnn_model(X_train_reshaped.shape[1:])
cnn_history = cnn_model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_val_reshaped, y_val))

# Train LSTM Model
lstm_model = build_lstm_model(X_train_reshaped.shape[1:])
lstm_history = lstm_model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_val_reshaped, y_val))

# Train Ensemble DNN Model
ensemble_dnn_model = build_ensemble_dnn_model()
ensemble_dnn_history = ensemble_dnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
ensemble_dnn_loss, ensemble_dnn_mse = ensemble_dnn_model.evaluate(X_val, y_val)
print(f"Ensemble DNN Loss: {ensemble_dnn_loss}, Ensemble DNN MSE: {ensemble_dnn_mse}")

In [ ]:
# Save the model
ensemble_dnn_model.save('/content/drive/MyDrive/SMAP_DOWNSCALING_CV/model')

Predicting in the new data at a resolution of 30m

In [ ]:
data_for_prediction = final_df_30m[['x', 'y' ,'VH', 'VV', 'CHIRPS','ET','DEM','SLOPE','TWI','NDVI','EVI','NDWI','SAVI','LAI','CROP','LC']]
data_for_prediction.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

X = data_for_prediction[['VV','VH','CHIRPS','ET','DEM','SLOPE' ,'TWI','NDVI','EVI','NDWI','SAVI','LAI','CROP','LC']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape X_scaled to add a dimension for channels
X_scaled_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

In [ ]:
#loading the trained model
import keras
model = keras.models.load_model('/content/drive/MyDrive/SMAP_DOWNSCALING_CV/model')

In [ ]:
#predict on the new data
y_pred = model.predict(X_scaled_reshaped)
y_pred.shape

In [ ]:
final_df_30m['SMAP'] = y_pred
final_df_30m = final_df_30m[['x', 'y', 'VH', 'VV','CHIRPS','ET','DEM','SLOPE','NDVI','EVI','NDWI','SAVI','LAI','CROP','LC', 'SMAP']]

In [ ]:
final_df_30m_new = final_df_30m[['x', 'y', 'SMAP']]
final_df_30m_new.head()

In [ ]:
# Determine the original rows and columns
original_rows = final_df_30m_new['y'].nunique()
original_cols = final_df_30m_new['x'].nunique()
original_smap = final_df_30m_new['SMAP'].nunique()

In [ ]:
import matplotlib.pyplot as plt

# Plotting the 'SMAP' values
plt.scatter(final_df_30m_new['x'], final_df_30m_new['y'], c=final_df_30m_new['SMAP'], cmap='viridis')
plt.colorbar(label='SMAP')

# Add labels and title to the plot
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('SMAP Predictions')

# Show the plot
plt.show()